# APIM ❤️ AI Foundry

## Secure Responses API lab
![flow](../../images/built-in-logging.gif)

Playground to try the [Azure OpenAI Responses API](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-secure) in a secure manner.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`... 


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the models and versions according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

aiservices_config = [{"name": "foundry1", "location": "swedencentral"}]

models_config = [{"name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 100}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}, 
                             {"name": "subscription2", "displayName": "Subscription 2"}, 
                             {"name": "subscription3", "displayName": "Subscription 3"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_name = "inference-api"  # name of the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

backend_id = 'foundry1' if len(aiservices_config) > 1 else aiservices_config[0]['name']
 
utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIName": { "value": inference_api_name },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key

<a id='requests'></a>
### 🧪 Test the Responses API using a direct HTTP call

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

In [ ]:
import json, requests, time

url = f"{apim_resource_gateway_url}/{inference_api_path}/openai/responses?api-version={inference_api_version}"
input = 'What is the OpenAI responses API?'

# Initialize a session for connection pooling and set any default headers
session = requests.Session()
session.headers.update({
    'api-key': api_key,
    'Content-Type': 'application/json'
})

try:
    response = session.post(url, json = {'model': models_config[0].get('name'), 'input': input})
    utils.print_response_code(response)
    print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")

    if (response.status_code == 200):
        data = json.loads(response.text)
        print(f"Model: {data['model']}")
        print(f"Token usage: {json.dumps(dict(data['usage']), indent = 4)}\n")
        print(f"Output: {json.dumps(data['output'], indent = 4)}\n")
    else:
        print(f"{response.text}\n")

finally:
    # Close the session to release the connection
    session.close()


<a id='sdk'></a>
### 🧪 Test the Responses API with the Azure OpenAI Python SDK


In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=api_key,
    base_url=f"{apim_resource_gateway_url}/{inference_api_path}/openai",
    default_query={"api-version": inference_api_version},
    api_version=inference_api_version
)

response = client.responses.create(   
  model=str(models_config[0].get('name')), 
  input="This is a test.",
)

print(response.model_dump_json(indent=2)) 

<a id='4'></a>
### 4️⃣ Update the policy so only the user who creates a response can see it.


In [ ]:
secure_policy_xml_file = "secure-policy.xml"

with open(secure_policy_xml_file, 'r') as file:
    policy_xml = file.read()
    policy_xml = policy_xml.replace('{backend-id}', backend_id)
    utils.update_api_policy(subscription_id, resource_group_name, apim_resource_name, inference_api_name, policy_xml)


<a id='testSecureWithDirectHttp'></a>
### 🧪 Test the Policy change with direct HTTP call

In this example, we demonstrate how the new APIM Policy enforces per-user access restrictions — meaning that only the user who created a response can view or use it later.

The code below:
- Obtains an Azure ARM access token to authenticate API requests.
- Creates two separate responses using two different simulated users (fishing-user and basketball-user).
  - For our example, we send in the userId as a header, but in production you would want to use the user's identity (e.g., from a JWT token). The APIM Policy we are using has this capability built-in, but it is commented out for testing purposes.
- Validates retrieval rules:
  - The basketball user can retrieve their own response (200 OK).
  - The fishing user attempting to retrieve the basketball user’s response receives a 403 Forbidden.
- Checks contextual linking:
  - The basketball user sends a follow-up request referencing their previous response (previous_response_id), and the API returns a result that incorporates the prior context.

This process confirms that the API:
- Correctly enforces ownership-based visibility for responses.
- Allows context chaining only for the original creator of a response.

In [ ]:
import json, requests, time

access_token = None

def pretty_out(resp):
    utils.print_response_code(response)
    print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")
    if (resp.status_code == 200):
        data = json.loads(resp.text)
        resp_id = data['id']
        print(f"Model: {data['model']}")
        print(f"Output: {json.dumps(data['output'], indent = 4)}")
        return resp_id
    else:
        print(f"{resp.text}\n")
        return None

# Get an ARM (management) access token via utils.run
output = utils.run( "az account get-access-token --resource https://management.azure.com/", "Retrieved access token", "Failed to retrieve access token")

if output.success and output.json_data:
    access_token = output.json_data.get("accessToken")
    expires_on = output.json_data.get("expiresOn")
    # Mask all but first 8 / last 6 chars
    if access_token:
        masked = f"{access_token[:8]}...{access_token[-6:]}"
        utils.print_info(f"Access token (masked): {masked}")
    utils.print_info(f"Expires On: {expires_on}")
else:
    utils.print_error("Could not fetch token")

baseUrl = f"{apim_resource_gateway_url}/{inference_api_path}/openai/responses"
queryParams = f"?api-version={inference_api_version}"
postUrl = f"{baseUrl}{queryParams}"

# Initialize a session for connection pooling and set any default headers
session = requests.Session()
session.headers.update({
    'api-key': api_key,
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token}' if access_token else ''
})

try:
    # 1) Create response as fishing user
    fishing_response_id = None
    session.headers['userId'] = 'fishing-user'
    fishing_payload = {
        'model': models_config[0].get('name'),
        'input': 'Hi, I like to fish'
    }
    response = session.post(postUrl, json = fishing_payload)
    utils.print_info("Fishing User Response - 200 expected:")
    fishing_response_id = pretty_out(response)
    print(f"Fishing User Response Id: {fishing_response_id}\n")

    # 2) Create response as basketball user
    basketball_response_id = None
    session.headers['userId'] = 'basketball-user'
    basketball_payload = {
        'model': models_config[0].get('name'),
        'input': 'Hi, I like to play basketball'
    }
    response = session.post(postUrl, json = basketball_payload)
    utils.print_info("Basketball User Response - 200 expected:")
    basketball_response_id = pretty_out(response)
    print(f"Basketball User Response Id: {basketball_response_id}\n")

    # 3) Get basketball user response as basketball user - should succeed with 200
    session.headers['userId'] = 'basketball-user'
    response = session.get(f"{baseUrl}/{basketball_response_id}{queryParams}")
    utils.print_info("Get Basketball User Response as Basketball User - 200 expected:")
    pretty_out(response)
    print(f"\n")

    # 4) Get basketball user response as fishing user - should fail with 403
    session.headers['userId'] = 'fishing-user'
    response = session.get(f"{baseUrl}/{basketball_response_id}{queryParams}")
    utils.print_info("Get Basketball User Response as Fishing User - 403 expected:")
    pretty_out(response)

    # 5) Post new response as basketball user to get context of previous response - should succeed with 200
    session.headers['userId'] = 'basketball-user'
    basketball_payload = {
        'model': models_config[0].get('name'),
        'input': 'What should I do this weekend?',
        'previous_response_id': basketball_response_id
    }
    response = session.post(postUrl, json = basketball_payload)
    utils.print_info("Basketball User Response - 200 expected, with a response that should include context of something to do with basketball:")
    basketball_response_id = pretty_out(response)
    print(f"Basketball User Response Id: {basketball_response_id}\n")


finally:
    # Close the session to release the connection
    session.close()

<a id='testSecureWithDirectHttp'></a>
### 🧪 Test the Policy Change with the Azure OpenAI Python SDK

Here we are doing the same example as above, except from the Python SDK. We demonstrate how the new APIM Policy enforces per-user access restrictions — meaning that only the user who created a response can view or use it later.

The code below:
- Obtains a access token to authenticate API requests.
- Creates two separate responses using two different simulated users (fishing-user and hiking-user).
  - For our example, we send in the userId as a header, but in production you would want to use the user's identity (e.g., from a JWT token). The APIM Policy we are using has this capability built-in, but it is commented out for testing purposes.
- Validates retrieval rules:
  - The hiking user can retrieve their own response (200 OK).
  - The fishing user attempting to retrieve the hiking user’s response receives a 403 Forbidden.
- Checks contextual linking:
  - The hiking user sends a follow-up request referencing their previous response (previous_response_id), and the API returns a result that incorporates the prior context.

This process confirms that the API:
- Correctly enforces ownership-based visibility for responses.
- Allows context chaining only for the original creator of a response.


In [ ]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Get an ARM (management) access token via get_bearer_token_provider
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://management.azure.com/.default")

client = AzureOpenAI(
    default_headers={'api-key': f'{api_key}'},
    base_url=f"{apim_resource_gateway_url}/{inference_api_path}/openai",
    azure_ad_token_provider=token_provider,
    api_version=inference_api_version
)

# 1) Create response as fishing user
fishing_response = client.responses.create(   
  model=str(models_config[0].get('name')), 
  input="Hi, I enjoy fishing.",
  extra_headers={'userId': 'fishing-user'}
)
print("Expected 200, with initial fishing response")
print(fishing_response.output) 

# 2) Create response as hiking user
hiking_response = client.responses.create(   
  model=str(models_config[0].get('name')), 
  input="Hi, I enjoy hiking.",
  extra_headers={'userId': 'hiking-user'}
)
print("Expected 200, with initial hiking response")
print(hiking_response.output) 

# 3) Get hiking user response as hiking user - should succeed with 200
try:
  hiking_as_hiking_response = client.responses.retrieve(
    hiking_response.id,
    extra_headers={'userId': 'hiking-user'}
  )
  print("Expected 200, with initial hiking response")
  print(hiking_as_hiking_response.output)
except Exception as e:
  print(f"Unexpected error: {e}")

# 4) Get hiking user response as fishing user - should fail with 403
try:
  hiking_as_fishing_response = client.responses.retrieve(
    hiking_response.id,
    extra_headers={'userId': 'fishing-user'}
  )
  print(hiking_as_fishing_response.output)
except Exception as e:
  print(f"Received 403 Forbidden as expected: {e}")

# 5) Post new response as hiking user to get context of previous response - should succeed with 200
try:
  hiking_response = client.responses.create(   
    model=str(models_config[0].get('name')), 
    previous_response_id=hiking_response.id,
    input="What should I do this weekend?",
    extra_headers={'userId': 'hiking-user'}
  )
  print("Expected 200, with output that has something to do with hiking for a weekend activity")
  print(hiking_response.output)
except Exception as e:
  print(f"Unexpected error: {e}")

<a id='kql'></a>
### 🔍 Display LLM logging


In [ ]:
import pandas as pd

query = "let llmHeaderLogs = ApiManagementGatewayLlmLog \
| where DeploymentName != ''; \
let llmLogsWithSubscriptionId = llmHeaderLogs \
| join kind=leftouter ApiManagementGatewayLogs on CorrelationId \
| project \
    SubscriptionId = ApimSubscriptionId, DeploymentName, TotalTokens; \
llmLogsWithSubscriptionId \
| summarize \
    SumTotalTokens      = sum(TotalTokens) \
  by SubscriptionId, DeploymentName"

output = utils.run(f"az monitor log-analytics query -w {log_analytics_id} --analytics-query \"{query}\"", "Retrieved log analytics query output", "Failed to retrieve log analytics query output") 
if output.success and output.json_data:
    table = output.json_data
    display(pd.DataFrame(table))


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.